In [1]:
import pandas as pd
import plotly.express as px
import altair as alt

In [2]:
readings = pd.read_csv('chemistry_readings.csv', parse_dates=['date'])
chemicals = pd.read_csv('added_chemicals.csv', parse_dates=['date'])
pinch = pd.read_csv('pinch_a_penny.csv', parse_dates=['date'])

### Notes
- w/ Total Alkalinity @ 80, the natural steady state pH is 8.2 
    - why is this a desired level when @ 40, things were steady state cool?
    - getting TA back down to 40 would take 1.08 gallons (!) of muriatic acid

In [3]:
readings.tail(3)

,date,time,total_hardness,total_chlorine,free_chlorine,ph_strip,total_alkalinity,stabilizer,ph_instrument,water_temp
127,2023-03-09,12:28,100.0,3.0,2.0,6.8,40.0,0.0,7.77,76.1
128,2023-03-14,13:30,250.0,1.0,1.0,6.8,40.0,0.0,7.99,74.4
129,2023-03-21,10:09,100.0,3.0,2.0,6.8,40.0,0.0,8.01,64.7


pH visualization

In [4]:
ph_cats = ['good', 'fix', 'rick']
ph_cat_colors = ['green', 'yellow', 'red']
ph_cat_breakpoints = [6.4, 6.8, 7.2, 7.8, 8.2, 8.6]

ph_ranges = pd.DataFrame([
    {'low': ph_cat_breakpoints[0], 'high': ph_cat_breakpoints[1], 'category': ph_cats[2]},
    {'low': ph_cat_breakpoints[1], 'high': ph_cat_breakpoints[2], 'category': ph_cats[1]},
    {'low': ph_cat_breakpoints[2], 'high': ph_cat_breakpoints[3], 'category': ph_cats[0]},
    {'low': ph_cat_breakpoints[3], 'high': ph_cat_breakpoints[4], 'category': ph_cats[1]},
    {'low': ph_cat_breakpoints[4], 'high': ph_cat_breakpoints[5], 'category': ph_cats[2]},
])

In [5]:
base = alt.Chart(readings.groupby('date', as_index=False).mean()).encode(
    x=alt.X('date:T', axis=alt.Axis(format='%m/%d'), title=None),
).properties(
    width=500,
    height=300
)

line = base.mark_line(color='#333').encode(
    alt.Y('ph_instrument:Q', title='pH')
)

points = line.mark_point(color='#333')

zones = alt.Chart(ph_ranges).mark_rect(opacity=.5).encode(
    y=alt.Y('low', scale=alt.Scale(domain=(6.5, 8.5))),
    y2='high',
    color=alt.Color('category:N', scale=alt.Scale(domain=ph_cats, range=ph_cat_colors))
)

# free_chlorine_line = base.mark_line(color='blue').encode(
#     alt.Y('free_chlorine:Q', title='Free Chlorine')
# )

zones + line + points

alt.LayerChart(...)

Chlorine visualization

In [6]:
fig = px.bar(
    readings.groupby('date', as_index=False).mean(),
    x = 'date',
    y = ['total_chlorine', 'free_chlorine'],
    barmode = 'group'
)

fig.show()

Calculate gallons added from fill time.

In [9]:
# enter how long the hose was on
duration_of_fill_min = 37
duration_of_fill_sec = 0

# caluclate amount of water added
fill_rate = 5/27  # emperically derived from sophisticated experimenation...it took 27s to fill a 5-gal. bucket
gallons_added = (duration_of_fill_min * 60 + duration_of_fill_sec) * fill_rate
print(f'You added {gallons_added:,.0f} gallons of water.')

You added 411 gallons of water.


In [8]:
chemicals.tail()

,date,time,chemical,amount,unit,note
102,2023-03-06,10:27,water,333.0,gal,30m 0s
103,2023-03-12,10:27,water,389.0,gal,35m 0s
104,2023-03-14,13:34,chlorine,1.0,"3"" puck",NaN
105,2023-03-17,14:38,water,367.0,gal,33m 0s
106,2023-03-21,10:11,chlorine,1.0,"3"" puck",NaN
